In [1]:
!python --version

Python 3.9.10


In [2]:
import ultralytics
ultralytics.__version__

'8.0.106'

In [4]:
import torch
torch.__version__

'2.0.1+cu117'

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

# Detect, track and count Persons

In [1]:
%cd yolov8_DeepSORT

D:\yolov8_SAM_env\yolov8_tracking\1_yolov8_DeepSORT\yolov8_DeepSORT


In [2]:
from ultralytics import YOLO

import time
import torch
import cv2 as cv
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model("images/person.jpg", save=True)



class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    probs = result.probs  # Class probabilities for classification outputs
    cls = boxes.cls.tolist()  # Convert tensor to list
    xyxy = boxes.xyxy
    xywh = boxes.xywh  # box with xywh format, (N, 4)
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)


image 1/1 D:\yolov8_SAM_env\yolov8_tracking\1_yolov8_DeepSORT\YOLOv8_DeepSORT\images\person.jpg: 384x640 1 person, 1 cup, 1 chair, 2 potted plants, 1 bed, 1 book, 67.0ms
Speed: 2.0ms preprocess, 67.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


[0.0, 58.0, 59.0, 58.0, 41.0, 73.0, 56.0]
Class: person
Class: potted plant
Class: bed
Class: potted plant
Class: cup
Class: book
Class: chair


# DeepSORT

In [3]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [4]:
# Define the video path
video_path = 'test1.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
frames = []

unique_track_ids = set()

In [2]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker
import cv2 as cv
import torch
import time
import ultralytics
from ultralytics import YOLO
import numpy as np

# Initialize DeepSort tracker
deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

# Define the video path
video_path = 'cars.mp4'
cap = cv.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define border lines
start_border_A = (0, 480)
end_border_A = (480, 480)
start_border_B = (512, 480)
end_border_B = (960, 480)

# Variables for counting cars
car_count_A = 0
car_count_B = 0

# Load YOLO model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        og_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        results = model(frame, device='cpu', classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13], conf=0.8)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        # Count cars as they cross the borders
        for track in tracker.tracker.tracks:
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            centroid = ((x1 + x2) / 2, y2)  # Calculate centroid of the bounding box
            
            # Check if centroid crosses border A
            if start_border_A[0] <= centroid[0] <= end_border_A[0] and centroid[1] >= start_border_A[1]:
                car_count_A += 1
            
            # Check if centroid crosses border B
            if start_border_B[0] <= centroid[0] <= end_border_B[0] and centroid[1] >= start_border_B[1]:
                car_count_B += 1

            # Draw bounding box and track ID
            cv.rectangle(og_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv.putText(og_frame, f"ID: {track.track_id}", (int(x1), int(y1) - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Display car counts
        cv.putText(og_frame, f"Car Count A: {car_count_A}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        cv.putText(og_frame, f"Car Count B: {car_count_B}", (10, 70), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

        out.write(cv.cvtColor(og_frame, cv.COLOR_RGB2BGR))
        og_frame = cv.resize(og_frame, (1920, 1080))
        cv.imshow("Video-mp4", og_frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv.destroyAllWindows()



0: 384x640 5 cars, 52.0ms
Speed: 2.0ms preprocess, 52.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.8ms
Speed: 4.2ms preprocess, 55.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.3ms
Speed: 2.3ms preprocess, 55.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.7ms
Speed: 2.5ms preprocess, 55.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 50.2ms
Speed: 2.8ms preprocess, 50.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 51.9ms
Speed: 3.3ms preprocess, 51.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 58.4ms
Speed: 2.7ms preprocess, 58.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 53.3ms
Speed: 3.3ms preprocess, 53.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

In [1]:
git init

SyntaxError: invalid syntax (2830201818.py, line 1)